In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
sys.path.append('..')  
import model_helper 
from helper import *
from plotly_helper import *

In [58]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    
    tree_df = pd.read_csv(f"../Cell_Population/{patient_id}_build_tree_posteriors.tsv", sep='\t')
    
    
    abundance_file = f"../Cell_Population/{patient_id}_cell_population_abundances.tsv"
    mcmc_file = f"../Cell_Population/{patient_id}_cell_population_mcmc_trace.tsv"  
    
    abundance_df = pd.read_csv(abundance_file, sep='\t')
    mcmc_df = pd.read_csv(mcmc_file, sep='\t')  
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count_sample, wbc_all,all_times





In [3]:
end_treatment_df = pd.read_csv("../CLL8_end_treatment.csv")

In [4]:
# Input patient information
# Input patient information
patient_id = 'GCLL-0164'
wbc_file = '../CLL8 CLL cell count 05222025 UPDATED.csv'
treatment_file = '../GCLL_treatment.txt'
UMI_start = 1
UMI_end = 4
treatment_start = 0


In [61]:
# Manually verify the sample list order is correct

In [5]:

sample_list = ['GCLL-0164-T-01', 'RP-1895_PDO-33508_SM-NKK1N_GCLL-0164-MRD-01_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-33508_SM-NKJZP_GCLL-0164-MRD-02_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-34670_SM-NPXSI_GCLL-0164-MRD-03_v1_Custom_Selection_OnPrem', 'GCLL-0164-T-02']

In [63]:
# Output file

In [6]:
outputfile_name = "CLL_subplot_combined_0164_wbc_updated.html"

Update patient specific information in the below cell 

In [7]:
# Get the treatment duration 

treatment_end = end_treatment_df[end_treatment_df.Patient == patient_id].Treatment_duration.iloc[0]

In [8]:
treatment_end

85

In [59]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [47]:
times_sample

[-11, 1220, 1302, 1479, 1668]

In [48]:
wbc_count

[98.2, 1.8, 3.1, 3.2, 3.1, 3.8, 3.4, 4.3, 5.6, 4.5, 5.9, 6.5, 18.9]

In [49]:
CLL_count_sample


[98.2, 0.06135248, 0.09879195, 0.52750545, 14.175]

In [50]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [60]:
tree_df

,n_iter,likelihood,edges
0,74,"[0.25000000000000083, 0.25000000000000083, 0.4...","1-2,1-3,3-5,1-4,None-1"
1,52,"[0.49999999999999906, 0.49999999999999906, 0.4...","1-2,1-3,2-4,3-5,None-1"
2,40,"[0.25000000000000083, 0.49999999999999906, 0.2...","1-2,1-3,2-5,3-4,None-1"
3,30,"[0.25000000000000083, 0.25000000000000083, 0.4...","1-2,1-3,2-4,2-5,None-1"
4,29,"[0.25000000000000083, 0.25000000000000083, 0.2...","1-2,1-3,3-4,3-5,None-1"
5,24,"[0.25000000000000083, 0.49999999999999906, 0.2...","1-2,1-3,2-5,5-4,None-1"


In [61]:
 plot_tree_html = plot_tree_plotly(tree_df, 0)

In [62]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [63]:
branch_annotations = {
    1: "del(13q14), CBLB_p.G739C",
    2: "del(5q32)", 
    3: "ATM_p.D2725N, ATM_p.T2947N, del(11q)",
    4: "ATM_p.Y2019C, ATM_p.R2598*, DIS3_p.D479G"
    
}

In [88]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

del(13q14)<br>CBLB_p.G739C
ATM_p.D2725N<br>ATM_p.T2947N<br>del(11q)
ATM_p.Y2019C<br>ATM_p.R2598*<br>DIS3_p.D479G


In [66]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [67]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [68]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [69]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [70]:
times_sample

[-11, 1220, 1302, 1479, 1668]

In [71]:
times_sample[extrapolate_start_idx:]

[1220, 1302, 1479, 1668]

In [72]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [73]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [74]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_35766/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
times_aft_tx

[85, 240, 517, 629, 723, 827, 931, 1030, 1126, 1220, 1302, 1479, 1668]

In [76]:
# Needs to manually check times_sliced_aft

In [77]:
times_sliced_aft

[517, 629, 723, 827, 931, 1030, 1126, 1220, 1302, 1479, 1668]

In [78]:
len(times_sliced_aft)

11

In [79]:
index_samples_model = slice(1,5)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [80]:
y

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 array([-7.23299079, -2.82194849, -7.53918101, -7.58666932, -8.93358591]),
 array([-8.00613999, -2.84873913, -3.30042065, -6.27384197, -6.29907335]),
 array([-5.85573332, -1.3667926 , -1.5480862 , -3.10185324, -4.39311821]),
 array([-3.46911949,  0.956857  ,  2.3094687 , -1.77192694,  0.26877723])]

In [81]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [82]:
wbc_model

[0.00209207,
 0.00241274,
 0.00297967,
 0.00505878,
 0.00667823,
 0.01091502,
 0.02364659,
 0.06135248,
 0.09879195,
 0.52750545,
 14.175]

In [83]:
len(wbc_model)

11

In [84]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [85]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [86]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [89]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL_subplot_combined_0164_wbc_updated.html
